In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
    .appName("dataeaze")\
    .master("local[2]")\
    .config("spark.sql.shuffle.partitions", "2")\
    .getOrCreate()

In [2]:
from pyspark.sql.types import StructField, StructType, StringType,LongType
custom_schema = StructType([
    StructField("Sr_No", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Startup_Name", StringType(), True),
    StructField("Industry_Vertical", StringType(), True),
    StructField("SubVertical", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Investors_Name", StringType(), True),
    StructField("InvestmentnType", StringType(), True),
    StructField("Amount_in_USD", LongType(), True),
    StructField("Remarks", StringType(), True),
])

In [3]:
df1 = spark.read.parquet("/home/sunbeam/Downloads/consumerInternet.parquet")
df1.show(truncate=False)

+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|Sr_No|Date      |Startup_Name|Industry_Vertical|SubVertical                                                                  |City      |Investors_Name                                                                       |InvestmentnType    |Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|152  |01/09/2018|Netmeds     |Consumer Internet|Online Pharmacy Chain                                                        |Chennai   |Sistema Asia Fund, Sistema JSFC and Tanncam Investment       

In [4]:
# df1.repartition(1).write.format('com.databricks.spark.csv').save("consumerinternet.csv",header = 'true')

In [5]:
dfx= spark.read.csv("consumerinternet.csv", header=True)
dfx.show(truncate=False)

+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|Sr_No|Date      |Startup_Name|Industry_Vertical|SubVertical                                                                  |City      |Investors_Name                                                                       |InvestmentnType    |Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|152  |01/09/2018|Netmeds     |Consumer Internet|Online Pharmacy Chain                                                        |Chennai   |Sistema Asia Fund, Sistema JSFC and Tanncam Investment       

In [6]:
df2= spark.read.csv("/home/sunbeam/Downloads/startup.csv", header=True)
df2.show(truncate=False)

+-----+----------+----------------------------+----------------------------+----------------------------------------+-------------+--------------------------------------------------------------------------------------+--------------------+-------------+-------+
|Sr_No|Date      |Startup_Name                |Industry_Vertical           |SubVertical                             |City         |Investors_Name                                                                        |InvestmentnType     |Amount_in_USD|Remarks|
+-----+----------+----------------------------+----------------------------+----------------------------------------+-------------+--------------------------------------------------------------------------------------+--------------------+-------------+-------+
|1    |09/01/2020|BYJU’S                      |E-Tech                      |E-learning                              |Bengaluru    |Tiger Global Management                                                            

In [7]:
mergeddf = df1.union(df2)
mergeddf.show(truncate=False)

+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|Sr_No|Date      |Startup_Name|Industry_Vertical|SubVertical                                                                  |City      |Investors_Name                                                                       |InvestmentnType    |Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+-----------------------------------------------------------------------------+----------+-------------------------------------------------------------------------------------+-------------------+-------------+-------+
|152  |01/09/2018|Netmeds     |Consumer Internet|Online Pharmacy Chain                                                        |Chennai   |Sistema Asia Fund, Sistema JSFC and Tanncam Investment       

In [8]:
mergeddf.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Startup_Name: string (nullable = true)
 |-- Industry_Vertical: string (nullable = true)
 |-- SubVertical: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Investors_Name: string (nullable = true)
 |-- InvestmentnType: string (nullable = true)
 |-- Amount_in_USD: string (nullable = true)
 |-- Remarks: string (nullable = true)



In [9]:
# How many startups are there in Pune City?
mergeddf.createOrReplaceTempView("startups")
punestartupcount = spark.sql('''select count(*) punestartupcount from startups where City="Pune"''')
punestartupcount.show()

+----------------+
|punestartupcount|
+----------------+
|             105|
+----------------+



In [10]:
# How many startups in Pune got their Seed/ Angel Funding?
puneangelcount = spark.sql('''select count(*) puneangelcount from startups where City = "Pune" and InvestmentnType = "Seed/ Angel Funding"''')
puneangelcount.show()

+--------------+
|puneangelcount|
+--------------+
|             2|
+--------------+



In [17]:
# What is the total amount raised by startups in Pune City? Hint - use regex_replace to get rid of null
totalpunefund = spark.sql('''select sum(regexp_replace(Amount_in_USD, '[^0-9]+', 0)) totalpunefund from startups where City="Pune"''')
totalpunefund.show()

+-------------+
|totalpunefund|
+-------------+
| 5.1318382E11|
+-------------+



In [19]:
# What are the top 5 Industry_Vertical which has the highest number of startups in India?
topfiv = spark.sql('''select Industry_Vertical, count(Startup_Name) sup_count from startups group by Industry_Vertical order by sup_count desc limit 5''')
topfiv.show()

+-----------------+---------+
|Industry_Vertical|sup_count|
+-----------------+---------+
|Consumer Internet|      941|
|       Technology|      478|
|        eCommerce|      186|
|              nan|      171|
|       Healthcare|       70|
+-----------------+---------+



In [20]:
# Find the top Investor(by amount) of each year.
some = spark.sql('''select Amount_in_USD from startups limit 5''')
some.show()

+-------------+
|Amount_in_USD|
+-------------+
|  3,50,00,000|
|    63,90,000|
|          N/A|
| 15,72,00,000|
| 10,00,00,000|
+-------------+



In [13]:
# Find the top startup(by amount raised) from each city?
topstartup = spark.sql('''select Amount_in_USD from startups where Amount_in_USD is not null limit 5''')
topstartup.show()

In [ ]:
# Which SubVertical had the highest growth(in number of startups) over the years?


In [ ]:
# Which SubVertical had the highest growth(in funding) over the years?